In [3]:
import csv
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option
import tomotopy as tp
import pandas as pd

origin_path = '201909.csv'
result_path = "201909_topic.csv"

origin=pd.read_csv(origin_path,encoding='utf-8')

text=origin['text']
print(text)

with open('datas9.txt','w',encoding='utf-8') as file:
    for item in text:
        file.write("%s\n" % item)

class Reader:
    def __init__(self, filePath):
        self.file = open(filePath,'r',encoding='UTF-8')
    
    def read(self, id):
        if id == 0: self.file.seek(0)
        return self.file.readline()

reader = Reader('datas9.txt')

file = open("한국어불용어100.txt", 'r', encoding="utf-8")
stopword=[]

while True:
    line = file.readline()
    if not line: break
    wordlist = line.split('\t')
    if (wordlist[1].startswith('N')):
        stopword.append(wordlist[0])

print(stopword)
filename="datas9.txt"  #파일 경로
file = open(filename, 'r', encoding="UTF-8")
kiwi=Kiwi()
kiwi.extract_add_words(reader.read)
kiwi.prepare()
stopwords = set(stopword)


def tokenize(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('N') and word not in stopwords] #불용어사전 적용


#LDA 모델을 적용해서 토픽 추출, k는 topic 개수
#alpha는 문헌-토픽 디리클레 분포의 하이퍼 파라미터
#eta는 토픽-단어 디리클레 분포의 하이퍼 파라미터 두개다 상수인듯하다
#min_cf는 단어의 최소 장서 빈도수, 전체 문헌내 출현빈도
#min_df는 단어의 최서 문헌 빈도수, 출현한 문헌 숫자수 의미
#tw는 용어 가중치 기법으로, ONE, IDF, PMI를 사용가능, ONE 보다는 PMI나 IDF 둘중 하나 사용 

file = open(result_path, "w", encoding='utf-8', newline = '')
writer = csv.writer(file)
writer.writerow(['index', 'topics'])
for i, line in enumerate(open(filename, encoding='utf-8')): #해당 경로의 파일을 받아와 한 라인씩 model에 추가
    model = tp.LDAModel(k=1, alpha=0.1, eta=0.001, min_cf=3,min_df=1, tw=tp.TermWeight.PMI)
    model.add_doc(tokenize(line)) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    s = ""
    for j in range(0,100):
        model.train(20) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        s = ', '.join(w for w, p in res)
    writer.writerow([i, s])
    # print(i, s)
file.close()
print("End")

topic = pd.read_csv(result_path, encoding='utf-8')

# print(csv)
# print(topic['topics'])

merge = pd.concat([origin, topic['topics']], axis=1)
print(merge)
merge.to_csv('201909_extract.csv', index=False, header=True,encoding="utf-8-sig")
print('merge topics')

0        예금이나 보험, 주식 계좌 등에 ‘잠자고 있는’ 개인의 금융재산이 8조원에 달하는 ...
1        어카운트인포·파인 접속하면 조회 가능 금감원 “금융사가 휴면재산 찾아 줘야”예금이나...
2        강변하수처리장 전경. 부산환경공단 제공부산환경공단이 하수처리장에서 발생하는 바이오가...
3        경남도가 ‘기술·창업·성장’ 선순환 혁신클러스터를 육성하는 강소연구개발특구(이하 ‘...
4        KB금융지주는 글로벌 창업기획사 ‘플러그 앤드 플레이’와 전략적 제휴협약(MOU)을...
                               ...                        
27275    인천시 ‘시민의 날’ 고베시장 등 초청 취소 / 경남도, 야마구치 단체와 공동포럼 ...
27276    홍콩 경찰의 불허 결정에도 열린 31일 ‘범죄인 인도 법안’(송환법) 반대 시위에서...
27277    "日 침략전쟁·식민지배 반성 안해"…"日정부, 개인청구권 부정한것 아냐""아베 '한...
27278    경남도 "격리 해제, 상황 종료"국내로 입국한 쿠웨이트 국적의 10대 여학생에게서 ...
27279    후반기 한미연합훈련이 종료됐지만 북한은 한반도 정세 긴장의 책임을 남측으로 돌리며 ...
Name: text, Length: 27280, dtype: object
['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']
End
             date press  category  \
0      2019/08/01  국민일보  경제,IT_과학   
1      20